In [1]:
import numpy as np
import pandas as pd
import requests
import re
from bs4 import BeautifulSoup
from tqdm import tqdm_notebook

## NAIC Scraping

In [2]:
url = 'https://classcodes.com/naics-code-list/'
r = requests.get(url)

r.raise_for_status()

html = r.text

In [3]:
soup = BeautifulSoup(html, 'html.parser')
p = soup.find_all('p')

The information of SOC codes and titles is contained in the text of `<p>`：

In [4]:
[x.get_text() for x in p][17:27]

['1) NAICS 111110 Soybean Farming',
 '2) NAICS 111120 Oilseed (except Soybean) Farming',
 '3) NAICS 111130 Dry Pea and Bean Farming',
 '4) NAICS 111140 Wheat Farming',
 '5) NAICS 111150 Corn Farming',
 '6) NAICS 111160 Rice Farming',
 '7) NAICS 111191 Oilseed and Grain Combination Farming',
 '8) NAICS 111199 All Other Grain Farming',
 '9) NAICS 111211 Potato Farming',
 '10) NAICS 111219 Other Vegetable (except Potato) and Melon Farming']

We can use regular expressions to extract the contents, and store them in naic_df:

In [5]:
naic_df = pd.DataFrame(columns=['NAIC Code', 'NAIC Title'])

code_re = re.compile(r'NAICS \d{6} .*')
for item in tqdm_notebook(p):
    try:
        naic_result = code_re.search(item.get_text()).group()
        naic_code = naic_result.split()[1]
        naic_title = ' '.join(naic_result.split()[2:]).strip().replace('’','\'')
        naic_df = naic_df.append([{'NAIC Code':int(naic_code), 'NAIC Title':naic_title}],ignore_index=True)
    except:
        continue

Now we can compare the data scraped from web with the file downloaded directly:

In [6]:
naic_dl_12 = pd.read_csv('dataset/6-digit_2012_Codes.csv')
naic_dl_12.columns = naic_df.columns
naic_dl_12.iloc[:,1] = naic_dl_12.iloc[:,1].apply(lambda x:x.strip().replace('’','\''))

After some naive parsing, we find that the scraping data are totally the same as the data in `6-digit_2012_Codes.csv`.

In [7]:
np.all(naic_df.values==naic_dl_12.values)

True

In [8]:
naic_df.to_pickle('dataset/naic.pkl')

## SOC Scraping

In [9]:
url = 'https://www.bls.gov/oes/current/oes_stru.htm'
r = requests.get(url)

r.raise_for_status()

html = r.text

In [10]:
soup = BeautifulSoup(html, 'html.parser')
li = soup.find_all('li')

The information of NAICS code and title is contained in the text of `<li>`：

In [11]:
for i, a in enumerate(tqdm_notebook(li)):
    print(i)
    print(a.get_text())

0
Home




Subject Areas»
Inflation & Prices
Spending & Time Use
Unemployment
Employment
Pay & Benefits
Productivity
Workplace Injuries
International
Demographics
Industries
Business Costs
Occupations
Geography



Resources For »
Business Leaders
Consumers
Developers
Economists
Investors
Jobseekers
Media
Public Policymakers
Students & Teachers
Survey Respondents
 
At a Glance Tables »
U.S. Economy
Regions, States, & Areas
Industries



MORE »
Announcements
BLS Speakers Available
Errata
Research
Response Rates
Statistical Sites




1
Subject Areas»
2
Inflation & Prices
3
Spending & Time Use
4
Unemployment
5
Employment
6
Pay & Benefits
7
Productivity
8
Workplace Injuries
9
International
10
Demographics
11
Industries
12
Business Costs
13
Occupations
14
Geography
15
Resources For »
16
Business Leaders
17
Consumers
18
Developers
19
Economists
20
Investors
21
Jobseekers
22
Media
23
Public Policymakers
24
Students & Teachers
25
Survey Respondents
26
 
27
At a Glance Tables »
28
U.S. Economy
2

            Moving Occupations
215
11-0000  Management Occupations
13-0000  Business and Financial
        Operations Occupations
15-0000  Computer and Mathematical
          Occupations
17-0000  Architecture and Engineering
             Occupations
19-0000  Life, Physical, and Social
           Science Occupations
21-0000  Community and Social Service
             Occupations
23-0000  Legal Occupations
25-0000  Education, Training, and
         Library Occupations
27-0000  Arts, Design, Entertainment,
             Sports, and Media Occupations
29-0000  Healthcare Practitioners and
             Technical Occupations
31-0000  Healthcare Support
        Occupations
33-0000  Protective Service
        Occupations
35-0000  Food Preparation and Serving
             Related Occupations
37-0000  Building and Grounds Cleaning
              and Maintenance Occupations
39-0000  Personal Care and Service
          Occupations
41-0000  Sales and Related
        Occupations
43-0000  Office and Admi

            Moving Occupations
223
27-0000  Arts, Design, Entertainment,
             Sports, and Media Occupations
29-0000  Healthcare Practitioners and
             Technical Occupations
31-0000  Healthcare Support
        Occupations
33-0000  Protective Service
        Occupations
35-0000  Food Preparation and Serving
             Related Occupations
37-0000  Building and Grounds Cleaning
              and Maintenance Occupations
39-0000  Personal Care and Service
          Occupations
41-0000  Sales and Related
        Occupations
43-0000  Office and Administrative
          Support Occupations
45-0000  Farming, Fishing, and Forestry
               Occupations
47-0000  Construction and Extraction
            Occupations
49-0000  Installation, Maintenance, and
               Repair Occupations
51-0000  Production Occupations
53-0000  Transportation and Material
            Moving Occupations
224
29-0000  Healthcare Practitioners and
             Technical Occupations
31-0000  Health


238
11-1000  Top Executives
        
11-1010  Chief Executives
          
11-1011  Chief Executives

11-1020  General and Operations Managers
          
11-1021  General and Operations
                   Managers 
11-1030  Legislators
          
11-1031  Legislators

11-2000  Advertising, Marketing, Promotions, Public Relations,
              and Sales Managers
        
11-2010  Advertising and Promotions Managers
          
11-2011  Advertising and Promotions
                       Managers 
11-2020  Marketing and Sales Managers
          
11-2021  Marketing Managers
11-2022  Sales Managers

11-2030  Public Relations and Fundraising Managers
          
11-2031  Public Relations and
                 Fundraising Managers 
11-3000  Operations Specialties Managers
        
11-3010  Administrative Services Managers
          
11-3011  Administrative Services
                    Managers 
11-3020  Computer and Information Systems Managers
          
11-3021  Computer and Information
      

                 Fundraising Managers 
249
11-2021  Marketing Managers
11-2022  Sales Managers

250
11-2022  Sales Managers

251
11-2030  Public Relations and Fundraising Managers
          
11-2031  Public Relations and
                 Fundraising Managers 
252
11-2031  Public Relations and
                 Fundraising Managers 
253
11-3000  Operations Specialties Managers
        
11-3010  Administrative Services Managers
          
11-3011  Administrative Services
                    Managers 
11-3020  Computer and Information Systems Managers
          
11-3021  Computer and Information
                     Systems Managers 
11-3030  Financial Managers
          
11-3031  Financial Managers

11-3050  Industrial Production Managers
          
11-3051  Industrial Production
                  Managers 
11-3060  Purchasing Managers
          
11-3061  Purchasing Managers

11-3070  Transportation, Storage, and Distribution Managers

          
11-3071  Transportation, Storage,
        

...

                     All Other 
1547
53-2010  Aircraft Pilots and Flight Engineers
          
53-2011  Airline Pilots, Copilots,
                      and Flight Engineers
53-2012  Commercial Pilots

53-2020  Air Traffic Controllers and Airfield Operations
              Specialists
          
53-2021  Air Traffic
            Controllers
53-2022  Airfield Operations
                Specialists 
53-2030  Flight Attendants
          
53-2031  Flight Attendants

1548
53-2011  Airline Pilots, Copilots,
                      and Flight Engineers
53-2012  Commercial Pilots

1549
53-2012  Commercial Pilots

1550
53-2020  Air Traffic Controllers and Airfield Operations
              Specialists
          
53-2021  Air Traffic
            Controllers
53-2022  Airfield Operations
                Specialists 
53-2030  Flight Attendants
          
53-2031  Flight Attendants

1551
53-2021  Air Traffic
            Controllers
53-2022  Airfield Operations
                Specialists 
1552
53-2022  Air

                    Services Drivers 
1564
53-3033  Light Truck or Delivery
                    Services Drivers 
1565
53-3040  Taxi Drivers and Chauffeurs
          
53-3041  Taxi Drivers and
             Chauffeurs 
53-3090  Miscellaneous Motor Vehicle Operators
          
53-3099  Motor Vehicle Operators,
                     All Other 
1566
53-3041  Taxi Drivers and
             Chauffeurs 
1567
53-3090  Miscellaneous Motor Vehicle Operators
          
53-3099  Motor Vehicle Operators,
                     All Other 
1568
53-3099  Motor Vehicle Operators,
                     All Other 
1569
53-4000  Rail Transportation Workers
        
53-4010  Locomotive Engineers and Operators
          
53-4011  Locomotive Engineers
53-4012  Locomotive Firers
53-4013  Rail Yard Engineers,
                 Dinkey Operators, and Hostlers 
53-4020  Railroad Brake, Signal, and Switch Operators

          
53-4021  Railroad Brake, Signal,
                    and Switch Operators 
53-4030  Railroad C

                     All Other 
1583
53-5010  Sailors and Marine Oilers
          
53-5011  Sailors and Marine
               Oilers 
53-5020  Ship and Boat Captains and Operators
          
53-5021  Captains, Mates, and
                 Pilots of Water Vessels
53-5022  Motorboat Operators

53-5030  Ship Engineers
          
53-5031  Ship Engineers

1584
53-5011  Sailors and Marine
               Oilers 
1585
53-5020  Ship and Boat Captains and Operators
          
53-5021  Captains, Mates, and
                 Pilots of Water Vessels
53-5022  Motorboat Operators

53-5030  Ship Engineers
          
53-5031  Ship Engineers

1586
53-5021  Captains, Mates, and
                 Pilots of Water Vessels
53-5022  Motorboat Operators

1587
53-5022  Motorboat Operators

1588
53-5030  Ship Engineers
          
53-5031  Ship Engineers

1589
53-5031  Ship Engineers

1590
53-6000  Other Transportation Workers
        
53-6010  Bridge and Lock Tenders
          
53-6011  Bridge and Lock
            

                     All Other 
1606
53-7010  Conveyor Operators and Tenders
          
53-7011  Conveyor Operators and
                   Tenders 
53-7020  Crane and Tower Operators
          
53-7021  Crane and Tower
            Operators 
53-7030  Dredge, Excavating, and Loading Machine Operators

          
53-7031  Dredge Operators
53-7032  Excavating and Loading
                   Machine and Dragline Operators
53-7033  Loading Machine Operators,
                       Underground Mining 
53-7040  Hoist and Winch Operators
          
53-7041  Hoist and Winch
            Operators 
53-7050  Industrial Truck and Tractor Operators
          
53-7051  Industrial Truck and
                 Tractor Operators 
53-7060  Laborers and Material Movers, Hand
          
53-7061  Cleaners of Vehicles and
                     Equipment
53-7062  Laborers and Freight,
                  Stock, and Material Movers, Hand
53-7063  Machine Feeders and
                Offbearers
53-7064  Packers and Pa

                     All Other 
1619
53-7061  Cleaners of Vehicles and
                     Equipment
53-7062  Laborers and Freight,
                  Stock, and Material Movers, Hand
53-7063  Machine Feeders and
                Offbearers
53-7064  Packers and Packagers,
                   Hand 
1620
53-7062  Laborers and Freight,
                  Stock, and Material Movers, Hand
53-7063  Machine Feeders and
                Offbearers
53-7064  Packers and Packagers,
                   Hand 
1621
53-7063  Machine Feeders and
                Offbearers
53-7064  Packers and Packagers,
                   Hand 
1622
53-7064  Packers and Packagers,
                   Hand 
1623
53-7070  Pumping Station Operators
          
53-7071  Gas Compressor and Gas
                   Pumping Station Operators
53-7072  Pump Operators, Except
                   Wellhead Pumpers
53-7073  Wellhead Pumpers

53-7080  Refuse and Recyclable Material Collectors
          
53-7081  Refuse and Recyclable
       

Again, we can use regular expressions to extract the contents, and store them in soc_df:

In [12]:
soc_df = pd.DataFrame(columns=['SOC Code', 'SOC Title'])

code_re = re.compile(r'\d{2}-\d{4}\D+')
for a in tqdm_notebook(li):
    soc_result = code_re.findall(a.get_text())
    for item in soc_result:
        soc_code = item.split()[0]
        soc_title = ' '.join(item.split()[1:]).strip()
        if soc_code not in soc_df.iloc[:,0].values: # avoid duplicates
            soc_df = soc_df.append([{'SOC Code':soc_code, 'SOC Title':soc_title}],ignore_index=True)

In [13]:
soc_df.to_pickle('dataset/soc.pkl')